# 🧹 Tratamento e Transformação de Tabelas
## 📌 Visão Geral

### Este notebook tem como objetivo padronizar, tratar e transformar os datasets que compõem o projeto de crédito.
### O processo é orientado pelas melhores práticas de Engenharia de Dados, garantindo qualidade, confiabilidade e rastreabilidade para suportar análises avançadas e modelos preditivos.

## 🎯 Objetivos

### Construção da Tabela Base
Estruturar e validar a tabela principal (train_base), assegurando a unicidade da chave primária, tratamento de valores nulos e conversão de tipos de dados.

### Integração e Tratamento de Tabelas Satélites
Aplicar funções padronizadas de avaliação, transformação e criação de variáveis derivadas, permitindo consistência na manipulação de diferentes datasets.

### Documentação Automatizada de Variáveis
Atualizar o dicionário de dados com descrições em português, assegurando maior clareza, interpretabilidade e alinhamento entre equipes de negócio e tecnologia.

## 🛠️ Benefícios

Confiabilidade dos Dados: identificação e tratamento de inconsistências, duplicidades e nulos.

Escalabilidade: uso de funções reutilizáveis para diferentes tabelas, reduzindo retrabalho e aumentando eficiência.

Transparência: documentação clara das regras aplicadas e exportação de relatórios para consulta.

Performance: persistência das tabelas tratadas em formato Parquet, otimizando consumo em pipelines analíticos.

## ✅ Entregáveis

Tabela base tratada e consolidada.

Tabelas satélites padronizadas, enriquecidas e integradas.

Dicionário de variáveis atualizado, em formato legível para consulta por stakeholders técnicos e de negócio.

In [1]:
# Criação da SparkSession
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ExemploSparkSession") \
    .getOrCreate()

In [2]:
# Importações
from pyspark.sql.functions import col, when, count, isnan, isnull, countDistinct, to_date, format_string, to_date, datediff, lit, coalesce, isnan
from pyspark.sql.types import FloatType, DoubleType
from datetime import datetime
import pandas as pd
import os
import glob

In [3]:
caminho = r"C:\Users\fred\meu_projeto_etl"

### 📌 Função `avaliar_dataset(df, pk=None)`

Essa função tem como objetivo **avaliar a qualidade e as características de um DataFrame Spark**.  
Ela gera um diagnóstico útil para entender a estrutura, a presença de valores nulos e a consistência dos dados.  

---

#### 🔎 O que a função faz:

1. **Resumo inicial**
   - Conta o número total de registros do DataFrame.  
   - Obtém o schema (nome e tipo de cada coluna).  

2. **Análise coluna a coluna**  
   Para cada coluna do DataFrame, calcula:  
   - **Tipo de dado** (ex.: string, integer, double).  
   - **% de valores nulos** e a **quantidade absoluta de nulos**.  
     - Para colunas numéricas (`float`, `double`), também verifica valores `NaN`.  
   - **Quantidade de valores distintos** (cardinalidade da coluna).  
   - **Exemplo de até 3 valores distintos não nulos** (útil para inspecionar rapidamente o conteúdo).  
   - Campo reservado para **observações** (pode ser usado futuramente para anotações manuais).  

   👉 Todos esses resultados são consolidados em um **DataFrame Pandas**, facilitando a visualização tabular no notebook.

3. **Resumo geral do dataset**  
   Retorna um dicionário com:  
   - Total de registros.  
   - Quantidade de colunas que possuem valores nulos.  
   - Quantidade de colunas totalmente nulas (100% nulos).  
   - Número de duplicatas na chave primária (se informada).  

4. **Verificação de chave primária (opcional)**  
   - Se o parâmetro `pk` for passado e a coluna existir:  
     - Conta o número de duplicatas na coluna que deveria ser única.  
     - Informa no console se a chave é válida (sem duplicatas) ou se há problemas.  
   - Caso a coluna informada não exista no DataFrame, emite um alerta.  

---

#### ✅ Exemplos de uso

```python
# Avaliação básica
resultados, resumo = avaliar_dataset(df)

# Avaliação com verificação de chave primária
resultados, resumo = avaliar_dataset(df, pk="id_cliente")

# Exibir resultados
display(resultados)  # tabela detalhada por coluna
print(resumo)        # resumo geral


In [4]:
def avaliar_dataset(df, pk=None):

    print("📊 Avaliação do DataFrame...")
    total_registros = df.count()
    schema = dict(df.dtypes)
    resultados = []

    for c in df.columns:
        tipo = schema[c]
        cond = isnull(col(c))
        if tipo in ("float", "double"):
            cond = cond | isnan(col(c))

        nulos = df.filter(cond).count()
        distintos = df.select(c).distinct().count()

        # Captura de exemplos distintos e não nulos
        try:
            exemplos_df = df.select(c).where(col(c).isNotNull()).distinct().limit(3)
            exemplo = exemplos_df.toPandas()[c].tolist()
        except:
            exemplo = []

        resultados.append({
            "coluna": c,
            "tipo": tipo,
            "% nulos": round(nulos / total_registros * 100, 2),
            "qtd_nulos": nulos,
            "valores_distintos": distintos,
            "exemplo_valores": exemplo,
            "observacoes": ""
        })

    resultados_df = pd.DataFrame(resultados)

    # 🔍 Resumo geral
    resumo = {
        "total_registros": total_registros,
        "colunas_com_nulos": resultados_df[resultados_df["qtd_nulos"] > 0].shape[0],
        "colunas_totalmente_nulas": resultados_df[resultados_df["qtd_nulos"] == total_registros].shape[0],
        "duplicatas_pk": None
    }

    # Verificação de chave primária
    if pk and pk in df.columns:
        duplicatas_pk = df.groupBy(pk).count().filter(col("count") > 1).count()
        resumo["duplicatas_pk"] = duplicatas_pk
        print(f"\n🔍 Chave primária '{pk}':")
        if duplicatas_pk > 0:
            print(f"⚠️ Encontradas {duplicatas_pk} duplicatas com base na chave '{pk}'")
        else:
            print("✅ Chave primária é única.")
    elif pk:
        print(f"⚠️ A coluna '{pk}' não está presente no DataFrame.")


    return resultados_df, resumo



### 📌 Função `criar_flags_nulos(df)`

Essa função tem como objetivo **criar colunas auxiliares ("flags") que indicam a presença de valores nulos ou `NaN` em cada coluna de um DataFrame Spark**.  

---

#### 🔎 O que a função faz:

1. **Iteração sobre as colunas do DataFrame**  
   Para cada coluna:
   - Define uma condição `cond` que identifica registros onde o valor é **nulo** (`isNull`) ou **NaN** (`isnan`).  

2. **Verificação de existência de valores ausentes**  
   - Se a coluna tiver ao menos um valor ausente:  
     - Cria uma **nova coluna flag** com o sufixo `_flag`.  
       - Valor `1` → quando o registro é nulo/NaN.  
       - Valor `0` → quando o registro é válido.  
     - Exemplo: se a coluna original for `idade`, será criada `idade_flag`.  
   - Caso não haja valores ausentes:  
     - Nenhuma coluna extra é criada.  
     - Um aviso é impresso no console informando que a coluna está 100% completa.  

3. **Retorno**  
   - O DataFrame atualizado com as colunas de flags adicionadas.  

---

#### ✅ Exemplo de uso

```python
# Executar a função
df_flags = criar_flags_nulos(df)

# Listar todas as flags criadas
flags_criadas = [c for c in df_flags.columns if c.endswith("_flag")]
print("Flags criadas:", flags_criadas)

# Se quiser inspecionar algumas flags específicas:
df_flags.select(flags_criadas).limit(5).toPandas()



In [5]:
def criar_flags_nulos(df):
    for c in df.columns:
        # Define a condição de ausência (null ou NaN)
        cond = col(c).isNull() | isnan(col(c))

        # Verifica se existe ao menos um valor ausente
        if df.filter(cond).count() > 0:
            nova_coluna = f"{c}_flag"
            df = df.withColumn(nova_coluna, when(cond, 1).otherwise(0))
            print(f"✅ Flag criada: {nova_coluna}")
        else:
            print(f"🟩 Sem valores ausentes em: {c} (flag não criada)")
    return df


### 📌 Função `detectar_e_converter_datas(df, formatos_tentados=None, max_amostras=None)`

Esta função tem como objetivo **detectar automaticamente colunas do tipo string que parecem representar datas** e convertê-las para o tipo `date` do PySpark.  
Ela é útil em datasets que chegam com campos de datas em formato textual e podem variar entre diferentes padrões (ex.: `yyyy-MM-dd`, `dd/MM/yyyy`, etc.).

---

#### 🔎 O que a função faz

1. **Parâmetros**
   - `df`: DataFrame Spark de entrada.
   - `formatos_tentados`: lista de formatos de data aceitos.  
     - Se não informado, usa um conjunto padrão que cobre formatos comuns no Brasil e no exterior.
   - `max_amostras`: número máximo de registros não nulos usados como amostra por coluna (default: 100).  
     - Evita trazer todo o dataset para o driver.

2. **Pré-processamento**
   - Considera apenas colunas do tipo `string`.  
   - Coleta até `max_amostras` valores não nulos da coluna, converte para string, remove espaços extras e transforma em lista.  
   - Se o valor contém horário (`2024-01-01 12:00:00` ou `2024-01-01T12:00:00`), apenas a parte da data é considerada.

3. **Validação de formatos**
   - Para cada formato da lista:
     - Tenta converter as amostras para `datetime` no Python (fazendo o mapeamento de tokens `yyyy → %Y`, `MM → %m`, `dd → %d`).  
     - Caso especial: formatos `yyyy-MM` são completados com `-01` (assumindo o primeiro dia do mês) apenas para validação.
     - Conta quantos valores foram parseados com sucesso e estão dentro de um intervalo plausível (anos entre 1950 e 2050).

4. **Critério de conversão**
   - Se **≥70% das amostras** de uma coluna forem válidas em algum formato, a função assume que esse é o padrão da coluna.  
   - A coluna é convertida no Spark com `to_date(coluna, formato)`.  
   - O nome da coluna é mantido (substituindo os valores originais).  
   - O processo é interrompido após o primeiro formato válido.

5. **Retorno**
   - Um novo DataFrame com as colunas convertidas para `date`.  
   - Uma lista com os nomes das colunas que foram convertidas.

---

#### ✅ Exemplo de uso

```python
df_convertido, colunas = detectar_e_converter_datas(df)

print("Colunas convertidas:", colunas)
df_convertido.printSchema()

# Visualizar uma amostra das colunas de data
df_convertido.select(colunas).show(10, truncate=False)


In [6]:
from pyspark.sql.functions import col, to_date

def detectar_e_converter_datas(df, formatos_tentados=None, max_amostras=None):
    """
    Detecta colunas string que parecem datas e converte para date.
    """
    # Define formatos padrão caso nenhum seja passado
    if formatos_tentados is None:
        formatos_tentados = [
            "yyyy-MM-dd",
            "dd/MM/yyyy",   # formato comum no Brasil
            "dd-MM-yyyy",
            "yyyy/MM/dd",
            "MM/dd/yyyy",
            "yyyy-MM",      # ano e mês apenas
            "yyyyMMdd",     # formato compacto
            "ddMMyyyy",
        ]
    # Define quantidade máxima de amostras por coluna
    if max_amostras is None:
        max_amostras = 100

    colunas_convertidas = []

    # Cache do schema (tipos de colunas)
    schema = dict(df.dtypes)

    for c in df.columns:
        # Só analisa colunas string
        if schema.get(c) != "string":
            continue

        # Coleta amostras não nulas da coluna
        try:
            amostras = (
                df.select(c)
                  .where(col(c).isNotNull())
                  .limit(max_amostras)
                  .toPandas()[c]
                  .dropna()
                  .astype(str)    # garante string
                  .str.strip()    # remove espaços extras
                  .tolist()
            )
        except Exception:
            amostras = []

        if not amostras:
            continue

        # Função auxiliar: remove parte de horário (ex.: "2024-01-01 12:00:00")
        def _limpo(v: str) -> str:
            if " " in v:
                v = v.split(" ", 1)[0]
            if "T" in v:
                v = v.split("T", 1)[0]
            return v

        amostras = [_limpo(v) for v in amostras if v]

        # Testa cada formato da lista
        for fmt in formatos_tentados:
            convertidos_validos = 0
            # Ajusta formato Spark -> formato Python
            py_fmt = fmt.replace("yyyy", "%Y").replace("MM", "%m").replace("dd", "%d")

            for valor in amostras:
                v = valor
                try:
                    # Caso especial: formato "yyyy-MM" precisa de dia para validar
                    if fmt == "yyyy-MM":
                        dt = datetime.strptime(v + "-01", "%Y-%m-%d")
                    else:
                        dt = datetime.strptime(v, py_fmt)

                    # Validação de faixa (anos plausíveis)
                    if 1950 <= dt.year <= 2050 and 1 <= dt.month <= 12 and 1 <= dt.day <= 31:
                        convertidos_validos += 1
                except Exception:
                    continue

            # Converte se >=70% das amostras forem válidas
            if convertidos_validos >= int(0.7 * len(amostras)):
                print(f"✅ Coluna '{c}' convertida com formato: {fmt}")
                df = df.withColumn(c, to_date(col(c), fmt))  # conversão Spark
                colunas_convertidas.append(c)
                break  # para de testar formatos após o primeiro sucesso

    return df, colunas_convertidas


## 1. 🧱 Tratamento da Tabela Base

In [7]:
#Importação da tabela base
train_base = spark.read.parquet(fr"{caminho}\\data\raw\train\train_base.parquet")

### Tratamento de nulos e duplicidade

In [8]:
# Função avaliar_dataset
df = train_base
nome_dataset = "train_base"

# Avaliação
tabela_resultado, resumo = avaliar_dataset(df=df, pk="case_id")

# Visualização
from IPython.display import display
display(tabela_resultado)
print(resumo)

# Nome e caminho do arquivo
nome_arquivo = f"{nome_dataset}_avaliar"
caminho_saida = fr"{caminho}\\docs\\{nome_arquivo}.csv"

# Salvar CSV
tabela_resultado.to_csv(caminho_saida, index=False, encoding="utf-8-sig")
print(f"✅ Tabela salva em: {caminho_saida}")


📊 Avaliação do DataFrame...

🔍 Chave primária 'case_id':
✅ Chave primária é única.


,coluna,tipo,% nulos,qtd_nulos,valores_distintos,exemplo_valores,observacoes
0,case_id,bigint,0.0,0,1526659,"[26, 29, 474]",
1,date_decision,string,0.0,0,644,"[2019-08-08, 2019-08-22, 2019-08-23]",
2,MONTH,bigint,0.0,0,22,"[201910, 201905, 202003]",
3,WEEK_NUM,bigint,0.0,0,92,"[26, 29, 65]",
4,target,bigint,0.0,0,2,"[0, 1]",


{'total_registros': 1526659, 'colunas_com_nulos': 0, 'colunas_totalmente_nulas': 0, 'duplicatas_pk': 0}
✅ Tabela salva em: C:\Users\fred\meu_projeto_etl\\docs\\train_base_avaliar.csv


### Tratamento de tipos de dados

In [9]:
# Alteração de tipo da coluna date_decision
train_base_date = train_base.withColumn('date_decision', to_date(col('date_decision'), 'yyyy-MM-dd'))

In [10]:
# Verificação da alteração do tipo
train_base_date.printSchema()

root
 |-- case_id: long (nullable = true)
 |-- date_decision: date (nullable = true)
 |-- MONTH: long (nullable = true)
 |-- WEEK_NUM: long (nullable = true)
 |-- target: long (nullable = true)



In [11]:
# Alteração de tipo da coluna MONTH
train_base_date = train_base_date.withColumn(
    "MONTH",
    to_date(format_string("%06d", col("MONTH")), "yyyyMM")
)

In [12]:
train_base_date.select('MONTH').distinct().orderBy('MONTH').show(20, truncate=False)

+----------+
|MONTH     |
+----------+
|2019-01-01|
|2019-02-01|
|2019-03-01|
|2019-04-01|
|2019-05-01|
|2019-06-01|
|2019-07-01|
|2019-08-01|
|2019-09-01|
|2019-10-01|
|2019-11-01|
|2019-12-01|
|2020-01-01|
|2020-02-01|
|2020-03-01|
|2020-04-01|
|2020-05-01|
|2020-06-01|
|2020-07-01|
|2020-08-01|
+----------+
only showing top 20 rows



In [13]:
# Verificação da alteração do tipo
train_base_date.printSchema()

root
 |-- case_id: long (nullable = true)
 |-- date_decision: date (nullable = true)
 |-- MONTH: date (nullable = true)
 |-- WEEK_NUM: long (nullable = true)
 |-- target: long (nullable = true)



In [14]:
# Monta o caminho completo até o diretório de saída
caminho_saida = os.path.join(caminho, "data", "interim", "train_base_tratada")

# Salva o DataFrame Spark em formato Parquet no local desejado
train_base_date.write.mode("overwrite").parquet(caminho_saida)


### 📌 Resumo do tratamento da tabela *Base*

- ✅ Após análise e verificação de **unicidade**, a coluna **`case_id`** foi definida como **chave primária**.  
- ✅ As **colunas de datas** foram identificadas e convertidas para o tipo **`date`**.  
- ✅ A tabela tratada foi salva em formato **Parquet**.


## 2. 🛰️  Tratamento das tabelas "Satélite"

### Tabela de descrição de variáveis

In [15]:
# Ler o arquivo markdown como texto
df_pd = pd.read_table(fr"{caminho}\\docs\tabela_descritiva.md", sep="|", engine="python", skipinitialspace=True)
df_pd = df_pd.loc[:, ~df_pd.columns.str.contains('^Unnamed')]
df_pd.columns = df_pd.columns.str.strip()
df_pd = df_pd.applymap(lambda x: x.strip() if isinstance(x, str) else x)

tabela_descritiva = spark.createDataFrame(df_pd)
tabela_descritiva.printSchema()
tabela_descritiva.show()

root
 |-- tabela: string (nullable = true)
 |-- coluna: string (nullable = true)
 |-- tipo_dado: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Descricao: string (nullable = true)

+--------------------+--------------------+-------------+--------------------+--------------------+
|              tabela|              coluna|    tipo_dado|         Description|           Descricao|
+--------------------+--------------------+-------------+--------------------+--------------------+
|:----------------...|:----------------...|:------------|:----------------...|:----------------...|
|          train_base|             case_id|       bigint|                 NaN|                 NaN|
|          train_base|       date_decision|       string|                 NaN|                 NaN|
|          train_base|               MONTH|       bigint|                 NaN|                 NaN|
|          train_base|            WEEK_NUM|       bigint|                 NaN|               

## Tratamento da tabela train_applprev_1_0

In [16]:
tabela_descritiva.columns

['tabela', 'coluna', 'tipo_dado', 'Description', 'Descricao']

In [17]:
# Filtrar e selecionar
tabela_descritiva_filtrada = tabela_descritiva.filter(
    col('tabela') == 'train_applprev_1_0'
).select('coluna', 'tipo_dado', 'Descricao')

# Mostrar para inspeção
tabela_descritiva_filtrada.show(45, truncate=False)

# Transformar em Pandas
tabela_descritiva_pd = tabela_descritiva_filtrada.toPandas()

# Salvar como Markdown
md_path = fr"{caminho}\\docs\tabela_descritiva1.md"

with open(md_path, "w", encoding="utf-8") as f:
    f.write(tabela_descritiva_pd.to_markdown(index=False))


+---------------------------+---------+--------------------------------------------------------------------------------------------------------+
|coluna                     |tipo_dado|Descricao                                                                                               |
+---------------------------+---------+--------------------------------------------------------------------------------------------------------+
|case_id                    |bigint   |NaN                                                                                                     |
|actualdpd_943P             |double   |Dias vencidos (DPD) do contrato anterior (real).                                                        |
|annuity_853A               |double   |Anuidade mensal para aplicativos anteriores.                                                            |
|approvaldate_319D          |string   |Data de aprovação do pedido anterior                                                       

In [18]:
# Carregando e unindo tabelas
train_applprev_1_0 = spark.read.parquet(fr"{caminho}\\data\raw\train\train_applprev_1_0.parquet")
train_applprev_1_1 = spark.read.parquet(fr"{caminho}\\data\raw\train\train_applprev_1_1.parquet")
train_applprev_1 = train_applprev_1_0.unionByName(train_applprev_1_1)

In [19]:
# Função avaliar_dataset
df = train_applprev_1
nome_dataset = "train_applprev_1"

# Avaliação
tabela_resultado, resumo = avaliar_dataset(df=df, pk="case_id")

# Visualização
from IPython.display import display
display(tabela_resultado)
print(resumo)

# Nome e caminho do arquivo
nome_arquivo = f"{nome_dataset}_avaliar"
caminho_saida = fr"{caminho}\\docs\\{nome_arquivo}.csv"

# Salvar CSV
tabela_resultado.to_csv(caminho_saida, index=False, encoding="utf-8-sig")
print(f"✅ Tabela salva em: {caminho_saida}")


📊 Avaliação do DataFrame...

🔍 Chave primária 'case_id':
⚠️ Encontradas 991955 duplicatas com base na chave 'case_id'


,coluna,tipo,% nulos,qtd_nulos,valores_distintos,exemplo_valores,observacoes
0,case_id,bigint,0.00,0,1221522,"[29, 964, 2453]",
1,actualdpd_943P,double,0.04,2500,190,"[160.0, 8.0, 70.0]",
2,annuity_853A,double,3.84,250736,89298,"[7171.0, 3069.6, 5482.2]",
3,approvaldate_319D,string,46.13,3010294,5403,"[2019-08-08, 2019-08-22, 2017-05-14]",
4,byoccupationinc_3656910L,double,76.49,4991531,29700,"[24923.0, 45953.0, 7542.62]",
5,cancelreason_3545846M,string,0.00,0,76,"[P205_40_167, P150_0_30, P60_137_164]",
6,childnum_21L,double,54.54,3559424,21,"[8.0, 0.0, 7.0]",
7,creationdate_885D,string,0.00,66,5406,"[2016-08-17, 2015-05-01, 2017-12-05]",
8,credacc_actualbalance_314A,double,95.10,6205945,93622,"[49.598003, 157.66801, 24594.0]",
9,credacc_credlmt_575A,double,2.97,194132,52332,"[14452.0, 24594.0, 16754.6]",


{'total_registros': 6525979, 'colunas_com_nulos': 32, 'colunas_totalmente_nulas': 0, 'duplicatas_pk': 991955}
✅ Tabela salva em: C:\Users\fred\meu_projeto_etl\\docs\\train_applprev_1_avaliar.csv


In [20]:
# Converter todas as colunas de data para DateType (assumindo formato yyyy-MM-dd)
train_applprev_1_date = train_applprev_1.withColumn("approvaldate_319D", to_date("approvaldate_319D")) \
       .withColumn("creationdate_885D", to_date("creationdate_885D")) \
       .withColumn("dateactivated_425D", to_date("dateactivated_425D")) \
       .withColumn("dtlastpmt_581D", to_date("dtlastpmt_581D")) \
       .withColumn("dtlastpmtallstes_3545839D", to_date("dtlastpmtallstes_3545839D")) \
       .withColumn("employedfrom_700D", to_date("employedfrom_700D")) \
       .withColumn("firstnonzeroinstldate_307D", to_date("firstnonzeroinstldate_307D"))


In [21]:
# Dias contados
train_applprev_1 = train_applprev_1_date.withColumn("dias_para_aprovacao", datediff("approvaldate_319D", "creationdate_885D")) \
       .withColumn("dias_ate_ativacao", datediff("dateactivated_425D", "creationdate_885D")) \
       .withColumn("dias_ult_pagamento", datediff("dtlastpmt_581D", "creationdate_885D")) \
       .withColumn("dias_ult_pagamento_all", datediff("dtlastpmtallstes_3545839D", "creationdate_885D")) \
       .withColumn("dias_desde_inicio_emprego", datediff("creationdate_885D", "employedfrom_700D")) \
       .withColumn("dias_para_primeira_parcela", datediff("firstnonzeroinstldate_307D", "creationdate_885D"))


In [22]:
# Criação de flags
train_applprev_1 = train_applprev_1 \
  .withColumn("sem_aprovacao_flag", when(col("approvaldate_319D").isNull(), 1).otherwise(0)) \
  .withColumn("sem_ativacao_flag", when(col("dateactivated_425D").isNull(), 1).otherwise(0)) \
  .withColumn("sem_pagamento_flag", when(col("dtlastpmt_581D").isNull(), 1).otherwise(0)) \
  .withColumn("sem_pagamento_total_flag", when(col("dtlastpmtallstes_3545839D").isNull(), 1).otherwise(0)) \
  .withColumn("sem_emprego_flag", when(col("employedfrom_700D").isNull(), 1).otherwise(0)) \
  .withColumn("sem_parcela_flag", when(col("firstnonzeroinstldate_307D").isNull(), 1).otherwise(0))


In [23]:
colunas_float = [
    "credamount_590A",
    "currdebt_94A",
    "outstandingdebt_522A",
    "mainoccupationinc_437A",
    "byoccupationinc_3656910L",
    "annuity_853A",
    "revolvingaccount_394A"
]

for c in colunas_float:
    train_applprev_1 = train_applprev_1.withColumn(c, col(c).cast("double"))


In [24]:
# Flags de nulo/cruzamento
train_applprev_1 = train_applprev_1 \
    .withColumn("mainoccupationinc_null_flag", when(col("mainoccupationinc_437A").isNull(), 1).otherwise(0)) \
    .withColumn("byoccupationinc_null_flag", when(col("byoccupationinc_3656910L").isNull(), 1).otherwise(0)) \
    .withColumn("tem_revolving_flag", when(col("revolvingaccount_394A").isNotNull(), 1).otherwise(0))



In [25]:
from pyspark.sql.functions import coalesce

train_applprev_1 = train_applprev_1.withColumn("divida_total",
                   coalesce(col("currdebt_94A"), lit(0)) + coalesce(col("outstandingdebt_522A"), lit(0)))


In [26]:
# Criação de flag
train_applprev_1 = train_applprev_1 \
  .withColumn("limite_cartao_credito_flag", when(col("approvaldate_319D").isNull(), 1).otherwise(0))

In [27]:
from pyspark.sql.functions import when

categoricas_com_nulos = [
    "credtype_587L",
    "familystate_726L",
    "inittransactioncode_279L"
]

for c in categoricas_com_nulos:
    train_applprev_1 = train_applprev_1.withColumn(
        c,
        when(col(c).isNull(), "Desconhecido").otherwise(col(c))
    )


In [28]:
from pyspark.sql.types import StringType

categoricas_todas = [
    "cancelreason_3545846M", "credtype_587L", "district_544M",
    "education_1138M", "familystate_726L", "inittransactioncode_279L",
    "postype_4733339M", "profession_152M", "rejectreason_755M", "rejectreasonclient_4145042M"
]

for c in categoricas_todas:
    train_applprev_1 = train_applprev_1.withColumn(c, col(c).cast(StringType()))


In [29]:
from pyspark.sql.functions import when, col

# 🔹 1. Tratamento de flags booleanas (transformar em 0/1)
train_applprev_1 = train_applprev_1 \
    .withColumn("isbidproduct_390L", when(col("isbidproduct_390L") == True, 1)
                                      .when(col("isbidproduct_390L") == False, 0)
                                      .otherwise(0)) \
    .withColumn("isdebitcard_527L", when(col("isdebitcard_527L") == True, 1)
                                     .when(col("isdebitcard_527L") == False, 0)
                                     .otherwise(0))

# 🔹 2. Tratamento de colunas de contagem (nulos → 0, cast para int)
cols_contagem = [
    "childnum_21L",
    "pmtnum_8L",
    "tenor_203L",
    "credacc_transactions_402L"
]

for c in cols_contagem:
    train_applprev_1 = train_applprev_1.withColumn(
        c,
        (when(col(c).isNull(), 0).otherwise(col(c))).cast("int")
    )

# 🔹 3. Tratamento de categorias discretas com valor numérico (nulos → -1, cast para int)
train_applprev_1 = train_applprev_1 \
    .withColumn("credacc_status_367L", (when(col("credacc_status_367L").isNull(), -1)
                                        .otherwise(col("credacc_status_367L"))).cast("int")) \
    .withColumn("status_219L", (when(col("status_219L").isNull(), -1)
                                .otherwise(col("status_219L"))).cast("int"))


In [30]:
from pyspark.sql.functions import col, when, lit, first, avg, collect_set

from pyspark.sql.functions import when, col

# Define as colunas que indicam histórico de crédito anterior
colunas_credito = [
    "credacc_actualbalance_314A",
    "credacc_maxhisbal_375A",
    "credacc_minhisbal_90A",
    "downpmt_134A",
    "maxdpdtolerance_577P"
]

# Cria uma condição para verificar se TODAS estão nulas
condicao_sem_credito = None
for c in colunas_credito:
    condicao_sem_credito = col(c).isNull() if condicao_sem_credito is None else (condicao_sem_credito & col(c).isNull())

# Cria a flag binária
train_applprev_1 = train_applprev_1.withColumn(
    "sem_historico_credito_flag",
    when(condicao_sem_credito, 1).otherwise(0)
)

In [31]:
# Categóricas: preenchimento de nulo e cast para string
colunas_categoricas = [
    "district_544M",
    "education_1138M",
    "profession_152M",
    "postype_4733339M",
    "rejectreason_755M",
    "rejectreasonclient_4145042M"
]

for c in colunas_categoricas:
    train_applprev_1 = train_applprev_1.withColumn(c, when(col(c).isNull(), "Desconhecido").otherwise(col(c)).cast("string"))

In [32]:
train_applprev_1.coalesce(1).write.mode("overwrite").parquet(fr"{caminho}\\data\interim\train_applprev_1.parquet")

### 📌 Resumo do tratamento da tabela *train_applprev_1*

- ✅ Os **datasets particionados** foram carregados em uma única variável, permitindo que o tratamento fosse feito de forma **unificada**.  
- ✅ A função de **avaliação** foi executada para observar o **comportamento e formato** dos dados.  
- ✅ As **colunas que apresentaram formato de data** foram convertidas para o tipo **`date`**.  
- ✅ Foi criada uma **coluna de tempo (em dias)** para comparação entre **datas iniciais e finais**.  
- ✅ Foram criadas **flags** para indicar a presença de **valores nulos**.  
- ✅ Foi criada uma coluna de **dívida total**.  
- ✅ Colunas que tinham valores **numéricos mas eram categóricas** foram convertidas para **string**.  
- ✅ A tabela tratada foi salva em formato **Parquet**.

## Tratamento da tabela train_applprev_2

In [33]:
train_applprev_2 = spark.read.parquet(fr"{caminho}\\data\raw\train\train_applprev_2.parquet")

In [34]:
# Função avaliar_dataset
df = train_applprev_2
nome_dataset = "train_applprev_2"

# Avaliação
tabela_resultado, resumo = avaliar_dataset(df=df, pk="case_id")

# Visualização
from IPython.display import display
display(tabela_resultado)
print(resumo)

# Nome e caminho do arquivo
nome_arquivo = f"{nome_dataset}_avaliar"
caminho_saida = fr"{caminho}\\docs\\{nome_arquivo}.csv"

# Salvar CSV
tabela_resultado.to_csv(caminho_saida, index=False, encoding="utf-8-sig")
print(f"✅ Tabela salva em: {caminho_saida}")



📊 Avaliação do DataFrame...

🔍 Chave primária 'case_id':
⚠️ Encontradas 1182243 duplicatas com base na chave 'case_id'


,coluna,tipo,% nulos,qtd_nulos,valores_distintos,exemplo_valores,observacoes
0,case_id,bigint,0.00,0,1221522,"[29, 964, 2453]",
1,cacccardblochreas_147M,string,0.78,109249,10,"[P19_60_110, P41_107_150, P33_145_161]",
2,conts_type_509L,string,17.01,2394056,10,"[EMPLOYMENT_PHONE, WHATSAPP, PRIMARY_MOBILE]",
3,credacc_cards_status_52L,string,97.57,13733404,7,"[UNCONFIRMED, INACTIVE, BLOCKED]",
4,num_group1,bigint,0.00,0,20,"[19, 0, 7]",
5,num_group2,bigint,0.00,0,12,"[0, 7, 6]",


{'total_registros': 14075487, 'colunas_com_nulos': 3, 'colunas_totalmente_nulas': 0, 'duplicatas_pk': 1182243}
✅ Tabela salva em: C:\Users\fred\meu_projeto_etl\\docs\\train_applprev_2_avaliar.csv


### 📌 Resumo do tratamento da tabela *train_applprev_2*

- ✅ A tabela foi **avaliada** com a função de profiling (`avaliar_dataset`).  
- ❌ **Não foi incorporada ao pipeline de tratamento**, devido à:  
  - Baixa relevância das informações.  
  - Impossibilidade de definição de uma chave primária.  


## Tratamento da tabela train_credit_bureau_a_1

In [35]:
train_credit_bureau_a_1 = spark.read.parquet(fr"{caminho}\\data\raw\train\train_credit_bureau_a_1_*.parquet")

In [36]:
# Filtrar e selecionar
tabela_descritiva_filtrada = tabela_descritiva.filter(
    col('tabela') == 'train_credit_bureau_a_1_0'
).select('coluna', 'tipo_dado', 'Descricao')

# Mostrar para inspeção
tabela_descritiva_filtrada.show(100, truncate=False)

# Transformar em Pandas
tabela_descritiva_pd = tabela_descritiva_filtrada.toPandas()

# Salvar como Markdown
md_path = fr"{caminho}\\docs\tabela_descritiva1.md"

with open(md_path, "w", encoding="utf-8") as f:
    f.write(tabela_descritiva_pd.to_markdown(index=False))

+-------------------------------+---------+-------------------------------------------------------------------------------------------+
|coluna                         |tipo_dado|Descricao                                                                                  |
+-------------------------------+---------+-------------------------------------------------------------------------------------------+
|case_id                        |bigint   |NaN                                                                                        |
|annualeffectiverate_199L       |double   |Taxa de juros dos contratos fechados.                                                      |
|annualeffectiverate_63L        |double   |Taxa de juros para os contratos ativos.                                                    |
|classificationofcontr_13M      |string   |Classificação do contrato ativo.                                                           |
|classificationofcontr_400M     |string   |Class

In [37]:
# Função avaliar_dataset
df = train_credit_bureau_a_1
nome_dataset = "train_credit_bureau_a_1"

# Avaliação
tabela_resultado, resumo = avaliar_dataset(df=df, pk="case_id")

# Visualização
from IPython.display import display
display(tabela_resultado)
print(resumo)

# Nome e caminho do arquivo
nome_arquivo = f"{nome_dataset}_avaliar"
caminho_saida = fr"{caminho}\\docs\\{nome_arquivo}.csv"

# Salvar CSV
tabela_resultado.to_csv(caminho_saida, index=False, encoding="utf-8-sig")
print(f"✅ Tabela salva em: {caminho_saida}")


📊 Avaliação do DataFrame...

🔍 Chave primária 'case_id':
⚠️ Encontradas 1386081 duplicatas com base na chave 'case_id'


,coluna,tipo,% nulos,qtd_nulos,valores_distintos,exemplo_valores,observacoes
0,case_id,bigint,0.00,0,1386273,"[964, 1697, 1806]",
1,annualeffectiverate_199L,double,95.44,15213904,6598,"[9.13, 46.15, 13.4]",
2,annualeffectiverate_63L,double,98.05,15629294,5654,"[17.56, 41.89, 9.13]",
3,classificationofcontr_13M,string,0.00,0,11,"[be7b251d, ea6782cc, 1cf4e481]",
4,classificationofcontr_400M,string,0.00,0,389,"[51590aa9, acba4f13, ffee884a]",
...,...,...,...,...,...,...,...
74,totalamount_996A,double,91.19,14536246,283564,"[7747.1743, 300000.0, 197134.4]",
75,totaldebtoverduevalue_178A,double,91.81,14635534,51535,"[7565.216, 1406.9241, 2043.5801]",
76,totaldebtoverduevalue_718A,double,92.00,14664995,951,"[40637.5, 2.4, 65.200005]",
77,totaloutstanddebtvalue_39A,double,91.81,14635534,1081141,"[91101.96, 3130.3062, 13.2699995]",


{'total_registros': 15940537, 'colunas_com_nulos': 66, 'colunas_totalmente_nulas': 0, 'duplicatas_pk': 1386081}
✅ Tabela salva em: C:\Users\fred\meu_projeto_etl\\docs\\train_credit_bureau_a_1_avaliar.csv


In [38]:
# Executar a função criar_flags_nulos
train_credit_bureau_a_1_flags = criar_flags_nulos(train_credit_bureau_a_1)

# Listar todas as flags criadas
flags_criadas = [c for c in train_credit_bureau_a_1_flags.columns if c.endswith("_flag")]
print("Flags criadas:", flags_criadas)

# Se quiser inspecionar algumas flags específicas:
train_credit_bureau_a_1_flags.select(flags_criadas).limit(5).toPandas()

🟩 Sem valores ausentes em: case_id (flag não criada)
✅ Flag criada: annualeffectiverate_199L_flag
✅ Flag criada: annualeffectiverate_63L_flag
🟩 Sem valores ausentes em: classificationofcontr_13M (flag não criada)
🟩 Sem valores ausentes em: classificationofcontr_400M (flag não criada)
🟩 Sem valores ausentes em: contractst_545M (flag não criada)
🟩 Sem valores ausentes em: contractst_964M (flag não criada)
✅ Flag criada: contractsum_5085717L_flag
✅ Flag criada: credlmt_230A_flag
✅ Flag criada: credlmt_935A_flag
✅ Flag criada: dateofcredend_289D_flag
✅ Flag criada: dateofcredend_353D_flag
✅ Flag criada: dateofcredstart_181D_flag
✅ Flag criada: dateofcredstart_739D_flag
✅ Flag criada: dateofrealrepmt_138D_flag
✅ Flag criada: debtoutstand_525A_flag
✅ Flag criada: debtoverdue_47A_flag
🟩 Sem valores ausentes em: description_351M (flag não criada)
✅ Flag criada: dpdmax_139P_flag
✅ Flag criada: dpdmax_757P_flag
✅ Flag criada: dpdmaxdatemonth_442T_flag
✅ Flag criada: dpdmaxdatemonth_89T_flag
✅ Fl

,annualeffectiverate_199L_flag,annualeffectiverate_63L_flag,contractsum_5085717L_flag,credlmt_230A_flag,credlmt_935A_flag,dateofcredend_289D_flag,dateofcredend_353D_flag,dateofcredstart_181D_flag,dateofcredstart_739D_flag,dateofrealrepmt_138D_flag,...,prolongationcount_599L_flag,refreshdate_3813885D_flag,residualamount_488A_flag,residualamount_856A_flag,totalamount_6A_flag,totalamount_996A_flag,totaldebtoverduevalue_178A_flag,totaldebtoverduevalue_718A_flag,totaloutstanddebtvalue_39A_flag,totaloutstanddebtvalue_668A_flag
0,1,1,1,1,0,0,1,1,0,1,...,1,1,1,0,1,1,1,1,1,1
1,1,1,1,1,1,0,1,1,0,1,...,1,1,1,1,1,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,1,1


In [39]:
# Executar a função detectar_e_converter_datas
train_credit_bureau_a_1_date, cols_conv = detectar_e_converter_datas(train_credit_bureau_a_1_flags)

print("Colunas convertidas:", cols_conv)

# Verificar schema após conversão
print(train_credit_bureau_a_1_date.printSchema())

# Exibir amostra das colunas convertidas (sem coletar tudo)
train_credit_bureau_a_1_date.select(*cols_conv).show(10, truncate=False)

✅ Coluna 'dateofcredend_289D' convertida com formato: yyyy-MM-dd
✅ Coluna 'dateofcredend_353D' convertida com formato: yyyy-MM-dd
✅ Coluna 'dateofcredstart_181D' convertida com formato: yyyy-MM-dd
✅ Coluna 'dateofcredstart_739D' convertida com formato: yyyy-MM-dd
✅ Coluna 'dateofrealrepmt_138D' convertida com formato: yyyy-MM-dd
✅ Coluna 'lastupdate_1112D' convertida com formato: yyyy-MM-dd
✅ Coluna 'lastupdate_388D' convertida com formato: yyyy-MM-dd
✅ Coluna 'numberofoverdueinstlmaxdat_148D' convertida com formato: yyyy-MM-dd
✅ Coluna 'numberofoverdueinstlmaxdat_641D' convertida com formato: yyyy-MM-dd
✅ Coluna 'overdueamountmax2date_1002D' convertida com formato: yyyy-MM-dd
✅ Coluna 'overdueamountmax2date_1142D' convertida com formato: yyyy-MM-dd
✅ Coluna 'refreshdate_3813885D' convertida com formato: yyyy-MM-dd
Colunas convertidas: ['dateofcredend_289D', 'dateofcredend_353D', 'dateofcredstart_181D', 'dateofcredstart_739D', 'dateofrealrepmt_138D', 'lastupdate_1112D', 'lastupdate_388

+------------------+------------------+--------------------+--------------------+--------------------+----------------+---------------+-------------------------------+-------------------------------+---------------------------+---------------------------+--------------------+
|dateofcredend_289D|dateofcredend_353D|dateofcredstart_181D|dateofcredstart_739D|dateofrealrepmt_138D|lastupdate_1112D|lastupdate_388D|numberofoverdueinstlmaxdat_148D|numberofoverdueinstlmaxdat_641D|overdueamountmax2date_1002D|overdueamountmax2date_1142D|refreshdate_3813885D|
+------------------+------------------+--------------------+--------------------+--------------------+----------------+---------------+-------------------------------+-------------------------------+---------------------------+---------------------------+--------------------+
|2020-08-06        |NULL              |NULL                |2018-08-06          |NULL                |2019-01-11      |NULL           |NULL                           |NU

In [40]:
train_credit_bureau_a_1 = train_credit_bureau_a_1_date
train_credit_bureau_a_1.coalesce(1).write.mode("overwrite").parquet(
    fr"{caminho}\\data\interim\train_bureau_a_1.parquet"
)

### 📌 Resumo do tratamento da tabela *train_credit_bureau_a_1*

- ✅ A função de **avaliação do dataset** foi executada.  
- ✅ A função de **criação de flags de nulos** foi aplicada, gerando diversas novas colunas.  
- ✅ A função de **detecção e conversão de datas** foi executada, convertendo várias colunas para o tipo `date`.  
- 🔎 Após avaliação detalhada das colunas e seus valores, **não foram necessárias novas alterações**.  
- 💾 O dataset tratado foi salvo em formato **Parquet**.  


## Tratamento da tabela train_credit_bureau_a_2

In [41]:
train_credit_bureau_a_2 = spark.read.parquet(fr"{caminho}\\data\raw\train\train_credit_bureau_a_2_*.parquet")

In [42]:
# Filtrar e selecionar
tabela_descritiva_filtrada = tabela_descritiva.filter(
    col('tabela') == 'train_credit_bureau_a_2_0'
).select('coluna', 'tipo_dado', 'Descricao')

# Mostrar para inspeção
tabela_descritiva_filtrada.show(100, truncate=False)

# Transformar em Pandas
tabela_descritiva_pd = tabela_descritiva_filtrada.toPandas()

# Salvar como Markdown
md_path = fr"{caminho}\\docs\tabela_descritiva_credit_bureau_a_2.md"

with open(md_path, "w", encoding="utf-8") as f:
    f.write(tabela_descritiva_pd.to_markdown(index=False))

+-------------------------------+---------+-----------------------------------------------------------------------------------------------------------------------------------------------+
|coluna                         |tipo_dado|Descricao                                                                                                                                      |
+-------------------------------+---------+-----------------------------------------------------------------------------------------------------------------------------------------------+
|case_id                        |bigint   |NaN                                                                                                                                            |
|collater_typofvalofguarant_298M|string   |Tipo de avaliação colateral (contrato ativo).                                                                                                  |
|collater_typofvalofguarant_407M|string   |Tipo de avaliação

In [43]:
# Função avaliar_dataset
df = train_credit_bureau_a_2
nome_dataset = "train_credit_bureau_a_2"

# Avaliação
tabela_resultado, resumo = avaliar_dataset(df=df, pk="case_id")

# Visualização
from IPython.display import display
display(tabela_resultado)
print(resumo)

# Nome e caminho do arquivo
nome_arquivo = f"{nome_dataset}_avaliar"
caminho_saida = fr"{caminho}\\docs\\{nome_arquivo}.csv"

# Salvar CSV
tabela_resultado.to_csv(caminho_saida, index=False, encoding="utf-8-sig")
print(f"✅ Tabela salva em: {caminho_saida}")

📊 Avaliação do DataFrame...

🔍 Chave primária 'case_id':
⚠️ Encontradas 1385287 duplicatas com base na chave 'case_id'


,coluna,tipo,% nulos,qtd_nulos,valores_distintos,exemplo_valores,observacoes
0,case_id,bigint,0.00,0,1385288,"[21899, 22129, 22165]",
1,collater_typofvalofguarant_298M,string,0.00,0,5,"[8fd95e4b, 9a0c095e, 06fb9ba8]",
2,collater_typofvalofguarant_407M,string,0.00,0,8,"[9276e4bb, 8fd95e4b, 9a0c095e]",
3,collater_valueofguarantee_1124L,double,98.54,185545117,83275,"[300000.0, 14307267.43, 3708826.34]",
4,collater_valueofguarantee_876L,double,96.34,181414202,148993,"[894952.08, 300000.0, 217000.0]",
5,collaterals_typeofguarante_359M,string,0.00,0,15,"[9276e4bb, 2fd21cf1, b1c5e678]",
6,collaterals_typeofguarante_669M,string,0.00,0,15,"[9276e4bb, 2fd21cf1, b1c5e678]",
7,num_group1,bigint,0.00,0,333,"[26, 29, 65]",
8,num_group2,bigint,0.00,0,101,"[26, 29, 19]",
9,pmts_dpd_1073P,double,81.25,152991152,4415,"[299.0, 934.0, 305.0]",


{'total_registros': 188298452, 'colunas_com_nulos': 10, 'colunas_totalmente_nulas': 0, 'duplicatas_pk': 1385287}
✅ Tabela salva em: C:\Users\fred\meu_projeto_etl\\docs\\train_credit_bureau_a_2_avaliar.csv


In [44]:
# Executar a função criar_flags_nulos
train_credit_bureau_a_2_flags = criar_flags_nulos(train_credit_bureau_a_2)

# Listar todas as flags criadas
flags_criadas = [c for c in train_credit_bureau_a_2_flags.columns if c.endswith("_flag")]
print("Flags criadas:", flags_criadas)

# Se quiser inspecionar algumas flags específicas:
train_credit_bureau_a_2_flags.select(flags_criadas).limit(5).toPandas()

🟩 Sem valores ausentes em: case_id (flag não criada)
🟩 Sem valores ausentes em: collater_typofvalofguarant_298M (flag não criada)
🟩 Sem valores ausentes em: collater_typofvalofguarant_407M (flag não criada)
✅ Flag criada: collater_valueofguarantee_1124L_flag
✅ Flag criada: collater_valueofguarantee_876L_flag
🟩 Sem valores ausentes em: collaterals_typeofguarante_359M (flag não criada)
🟩 Sem valores ausentes em: collaterals_typeofguarante_669M (flag não criada)
🟩 Sem valores ausentes em: num_group1 (flag não criada)
🟩 Sem valores ausentes em: num_group2 (flag não criada)
✅ Flag criada: pmts_dpd_1073P_flag
✅ Flag criada: pmts_dpd_303P_flag
✅ Flag criada: pmts_month_158T_flag
✅ Flag criada: pmts_month_706T_flag
✅ Flag criada: pmts_overdue_1140A_flag
✅ Flag criada: pmts_overdue_1152A_flag
✅ Flag criada: pmts_year_1139T_flag
✅ Flag criada: pmts_year_507T_flag
🟩 Sem valores ausentes em: subjectroles_name_541M (flag não criada)
🟩 Sem valores ausentes em: subjectroles_name_838M (flag não criada

,collater_valueofguarantee_1124L_flag,collater_valueofguarantee_876L_flag,pmts_dpd_1073P_flag,pmts_dpd_303P_flag,pmts_month_158T_flag,pmts_month_706T_flag,pmts_overdue_1140A_flag,pmts_overdue_1152A_flag,pmts_year_1139T_flag,pmts_year_507T_flag
0,0,1,1,1,0,1,1,1,0,1
1,1,1,1,1,0,1,1,1,0,1
2,1,1,1,1,0,1,1,1,0,1
3,1,1,1,1,0,1,1,1,0,1
4,1,1,1,1,0,1,1,1,0,1


In [45]:
train_credit_bureau_a_2 = train_credit_bureau_a_2_flags
train_credit_bureau_a_2.coalesce(1).write.mode("overwrite").parquet(
    fr"{caminho}\\data\interim\train_bureau_a_2.parquet"
)

### 📌 Resumo do tratamento da tabela *train_credit_bureau_a_2*

- ✅ Execução da função de **avaliação do dataset**.  
- ✅ **Avaliação visual** do dataset.  
- ✅ Execução da função de **criação de flags** (valores nulos).  
- 💾 Dataset salvo em formato **Parquet**.  
  


## Tratamento da tabela train_credit_bureau_b_1

In [46]:
train_credit_bureau_b_1 = spark.read.parquet(fr"{caminho}\\data\raw\train\train_credit_bureau_b_1.parquet")

In [47]:
# Filtrar e selecionar
tabela_descritiva_filtrada = tabela_descritiva.filter(
    col('tabela') == 'train_credit_bureau_b_1'
).select('coluna', 'tipo_dado', 'Descricao')

# Mostrar para inspeção
tabela_descritiva_filtrada.show(100, truncate=False)

# Transformar em Pandas
tabela_descritiva_pd = tabela_descritiva_filtrada.toPandas()

# Salvar como Markdown
md_path = fr"{caminho}\\docs\tabela_descritiva_credit_bureau_b_1.md"

with open(md_path, "w", encoding="utf-8") as f:
    f.write(tabela_descritiva_pd.to_markdown(index=False))

+------------------------------+---------+------------------------------------------------------------------------------------------------------------------+
|coluna                        |tipo_dado|Descricao                                                                                                         |
+------------------------------+---------+------------------------------------------------------------------------------------------------------------------+
|case_id                       |bigint   |NaN                                                                                                               |
|amount_1115A                  |double   |Valor de crédito do contrato ativo fornecido pelo Credit Bureau.                                                  |
|classificationofcontr_1114M   |string   |Classificação do contrato ativo.                                                                                  |
|contractdate_551D             |string   |Data do co

In [48]:
# Função avaliar_dataset
df = train_credit_bureau_b_1
nome_dataset = "train_credit_bureau_b_1"

# Avaliação
tabela_resultado, resumo = avaliar_dataset(df=df, pk="case_id")

# Visualização
from IPython.display import display
display(tabela_resultado)
print(resumo)

# Nome e caminho do arquivo
nome_arquivo = f"{nome_dataset}_avaliar"
caminho_saida = fr"{caminho}\\docs\\{nome_arquivo}.csv"

# Salvar CSV
tabela_resultado.to_csv(caminho_saida, index=False, encoding="utf-8-sig")
print(f"✅ Tabela salva em: {caminho_saida}")

📊 Avaliação do DataFrame...

🔍 Chave primária 'case_id':
⚠️ Encontradas 27670 duplicatas com base na chave 'case_id'


,coluna,tipo,% nulos,qtd_nulos,valores_distintos,exemplo_valores,observacoes
0,case_id,bigint,0.00,0,36500,"[13248, 32954, 38996]",
1,amount_1115A,double,49.08,42110,20527,"[300000.0, 16596.0, 386322.4]",
2,classificationofcontr_1114M,string,0.00,0,10,"[ea6782cc, 436d55c2, 1cf4e481]",
3,contractdate_551D,string,4.54,3892,4076,"[2017-05-14, 2017-12-05, 2019-08-23]",
4,contractmaturitydate_151D,string,4.75,4079,4538,"[2019-08-08, 2019-08-23, 2020-02-26]",
5,contractst_516M,string,0.00,0,15,"[83931972, dd67cff0, 54132f86]",
6,contracttype_653M,string,0.00,0,25,"[1c9c5356, f4e17141, 60e784d6]",
7,credlmt_1052A,double,67.85,58210,9635,"[330000.0, 300000.0, 102197.6]",
8,credlmt_228A,double,81.20,69661,3536,"[20000.717, 300000.0, 79991.805]",
9,credlmt_3940954A,double,55.45,47573,9660,"[300000.0, 35734.0, 34401.402]",


{'total_registros': 85791, 'colunas_com_nulos': 35, 'colunas_totalmente_nulas': 0, 'duplicatas_pk': 27670}
✅ Tabela salva em: C:\Users\fred\meu_projeto_etl\\docs\\train_credit_bureau_b_1_avaliar.csv


In [49]:
train_credit_bureau_b_1.select('maxdebtpduevalodued_3940955A').show(30)

+----------------------------+
|maxdebtpduevalodued_3940955A|
+----------------------------+
|                         0.0|
|                        NULL|
|                        NULL|
|                         0.0|
|                         0.0|
|                         5.8|
|                         0.4|
|                        NULL|
|                       344.0|
|                        NULL|
|                       272.6|
|                         0.0|
|                   5.2000003|
|                        NULL|
|                        68.6|
|                         0.0|
|                         0.0|
|                         0.0|
|                         0.2|
|                         0.0|
|                        NULL|
|                         0.0|
|                    9.400001|
|                   6.2000003|
|                         0.0|
|                         0.0|
|                         0.0|
|                       162.6|
|                       260.0|
|       

In [50]:
from pyspark.sql import functions as F

COL = "maxdebtpduevalodued_3940955A"  # coluna de dias
MAX_DIAS_PLAUSIVEL = 2000             # ajuste se necessário

# 1) Filtrar valores absurdos (negativos ou > limite)
absurdos_df = (
    train_credit_bureau_b_1
    .filter((F.col(COL) < 0) | (F.col(COL) > F.lit(MAX_DIAS_PLAUSIVEL)))
)

qtd_absurdos = absurdos_df.count()

if qtd_absurdos == 0:
    # 2) Converter para inteiro (arredondando)
    train_credit_bureau_b_1 = train_credit_bureau_b_1.withColumn(COL, F.round(F.col(COL)).cast("int"))
    print(f"✅ Sem valores absurdos detectados. Coluna '{COL}' convertida para int.")
else:
    print(f"⚠️ Encontrados {qtd_absurdos} valores absurdos em '{COL}'. Exemplos:")
    absurdos_df.select(COL).show(20, truncate=False)  # mostra até 20 exemplos


⚠️ Encontrados 4 valores absurdos em 'maxdebtpduevalodued_3940955A'. Exemplos:
+----------------------------+
|maxdebtpduevalodued_3940955A|
+----------------------------+
|147470.61                   |
|147448.8                    |
|147448.8                    |
|147470.61                   |
+----------------------------+



In [51]:
train_credit_bureau_b_1 = train_credit_bureau_b_1.withColumn(
    COL,
    F.when((F.col(COL) < 0) | (F.col(COL) > MAX_DIAS_PLAUSIVEL), None).otherwise(F.col(COL))
)
print("✅ Valores absurdos substituídos por nulo.")


✅ Valores absurdos substituídos por nulo.


In [52]:
from pyspark.sql import functions as F

COL = "maxdebtpduevalodued_3940955A"  # coluna de dias
MAX_DIAS_PLAUSIVEL = 2000             # ajuste se necessário

# 1) Filtrar valores absurdos (negativos ou > limite)
absurdos_df = (
    train_credit_bureau_b_1
    .filter((F.col(COL) < 0) | (F.col(COL) > F.lit(MAX_DIAS_PLAUSIVEL)))
)

qtd_absurdos = absurdos_df.count()

if qtd_absurdos == 0:
    # 2) Converter para inteiro (arredondando)
    train_credit_bureau_b_1 = train_credit_bureau_b_1.withColumn(COL, F.round(F.col(COL)).cast("int"))
    print(f"✅ Sem valores absurdos detectados. Coluna '{COL}' convertida para int.")
else:
    print(f"⚠️ Encontrados {qtd_absurdos} valores absurdos em '{COL}'. Exemplos:")
    absurdos_df.select(COL).show(20, truncate=False)  # mostra até 20 exemplos


✅ Sem valores absurdos detectados. Coluna 'maxdebtpduevalodued_3940955A' convertida para int.


In [53]:
from pyspark.sql import functions as F

COL = "overdueamountmaxdateyear_432T"

df_check = (
    train_credit_bureau_b_1
    .select(
        F.count(F.when(F.col(COL).isNull(), 1)).alias("qtd_nulos"),
        F.count(F.when(F.col(COL) == 1900, 1)).alias("qtd_1900")
    )
)

df_check.show(truncate=False)


+---------+--------+
|qtd_nulos|qtd_1900|
+---------+--------+
|4567     |1       |
+---------+--------+



In [54]:
from pyspark.sql import functions as F

COL = "overdueamountmaxdateyear_432T"

anos_invalidos = (
    train_credit_bureau_b_1
    .filter((F.col(COL) < 1950) | (F.col(COL) > 2025))
    .groupBy(COL)
    .count()
    .orderBy("count", ascending=False)
)

anos_invalidos.show(truncate=False)


+-----------------------------+-----+
|overdueamountmaxdateyear_432T|count|
+-----------------------------+-----+
|1900.0                       |1    |
+-----------------------------+-----+



In [55]:
from pyspark.sql import functions as F

COL = "overdueamountmaxdateyear_432T"

train_credit_bureau_b_1 = train_credit_bureau_b_1.withColumn(
    COL,
    F.when(F.col(COL) == 1900, None).otherwise(F.col(COL))
)

print(f"✅ Valores 1900 em '{COL}' foram substituídos por NULL.")


✅ Valores 1900 em 'overdueamountmaxdateyear_432T' foram substituídos por NULL.


In [56]:
# Executar a função criar_flags_nulos
train_credit_bureau_b_1_flags = criar_flags_nulos(train_credit_bureau_b_1)

# Listar todas as flags criadas
flags_criadas = [c for c in train_credit_bureau_b_1_flags.columns if c.endswith("_flag")]
print("Flags criadas:", flags_criadas)

# Se quiser inspecionar algumas flags específicas:
train_credit_bureau_b_1_flags.select(flags_criadas).limit(5).toPandas()

🟩 Sem valores ausentes em: case_id (flag não criada)
✅ Flag criada: amount_1115A_flag
🟩 Sem valores ausentes em: classificationofcontr_1114M (flag não criada)
✅ Flag criada: contractdate_551D_flag
✅ Flag criada: contractmaturitydate_151D_flag
🟩 Sem valores ausentes em: contractst_516M (flag não criada)
🟩 Sem valores ausentes em: contracttype_653M (flag não criada)
✅ Flag criada: credlmt_1052A_flag
✅ Flag criada: credlmt_228A_flag
✅ Flag criada: credlmt_3940954A_flag
🟩 Sem valores ausentes em: credor_3940957M (flag não criada)
✅ Flag criada: credquantity_1099L_flag
✅ Flag criada: credquantity_984L_flag
✅ Flag criada: debtpastduevalue_732A_flag
✅ Flag criada: debtvalue_227A_flag
✅ Flag criada: dpd_550P_flag
✅ Flag criada: dpd_733P_flag
✅ Flag criada: dpdmax_851P_flag
✅ Flag criada: dpdmaxdatemonth_804T_flag
✅ Flag criada: dpdmaxdateyear_742T_flag
✅ Flag criada: installmentamount_644A_flag
✅ Flag criada: installmentamount_833A_flag
✅ Flag criada: instlamount_892A_flag
✅ Flag criada: inter

,amount_1115A_flag,contractdate_551D_flag,contractmaturitydate_151D_flag,credlmt_1052A_flag,credlmt_228A_flag,credlmt_3940954A_flag,credquantity_1099L_flag,credquantity_984L_flag,debtpastduevalue_732A_flag,debtvalue_227A_flag,...,overdueamountmaxdateyear_432T_flag,periodicityofpmts_997L_flag,periodicityofpmts_997M_flag,pmtdaysoverdue_1135P_flag,pmtnumpending_403L_flag,residualamount_1093A_flag,residualamount_127A_flag,residualamount_3940956A_flag,totalamount_503A_flag,totalamount_881A_flag
0,0,0,0,1,1,1,1,1,0,0,...,0,1,0,0,0,1,1,1,1,1
1,1,0,0,0,0,0,0,0,1,1,...,1,1,0,1,1,0,0,1,0,0
2,1,0,0,1,1,0,0,0,1,1,...,1,1,0,1,1,1,1,1,0,0
3,0,0,0,1,1,1,1,1,0,0,...,0,1,0,0,0,1,1,1,1,1
4,0,0,0,1,1,1,1,1,0,0,...,0,1,0,0,0,1,1,1,1,1


In [57]:
# Executar a função detectar_e_converter_datas
train_credit_bureau_b_1_date, cols_conv = detectar_e_converter_datas(train_credit_bureau_b_1_flags)

print("Colunas convertidas:", cols_conv)

# Verificar schema após conversão
print(train_credit_bureau_b_1_date.printSchema())

# Exibir amostra das colunas convertidas (sem coletar tudo)
train_credit_bureau_b_1_date.select(*cols_conv).show(10, truncate=False)

✅ Coluna 'contractdate_551D' convertida com formato: yyyy-MM-dd
✅ Coluna 'contractmaturitydate_151D' convertida com formato: yyyy-MM-dd
✅ Coluna 'lastupdate_260D' convertida com formato: yyyy-MM-dd
Colunas convertidas: ['contractdate_551D', 'contractmaturitydate_151D', 'lastupdate_260D']
root
 |-- case_id: long (nullable = true)
 |-- amount_1115A: double (nullable = true)
 |-- classificationofcontr_1114M: string (nullable = true)
 |-- contractdate_551D: date (nullable = true)
 |-- contractmaturitydate_151D: date (nullable = true)
 |-- contractst_516M: string (nullable = true)
 |-- contracttype_653M: string (nullable = true)
 |-- credlmt_1052A: double (nullable = true)
 |-- credlmt_228A: double (nullable = true)
 |-- credlmt_3940954A: double (nullable = true)
 |-- credor_3940957M: string (nullable = true)
 |-- credquantity_1099L: double (nullable = true)
 |-- credquantity_984L: double (nullable = true)
 |-- debtpastduevalue_732A: double (nullable = true)
 |-- debtvalue_227A: double (nul

In [58]:
train_credit_bureau_b_1 = train_credit_bureau_b_1_date
train_credit_bureau_b_1.coalesce(1).write.mode("overwrite").parquet(
    fr"{caminho}\\data\interim\train_bureau_b_1.parquet"
)

### 📌 Resumo do tratamento da tabela *train_credit_bureau_b_1*

- ✅ Execução da função para **avaliar dataset**.  
- ✅ **Avaliação visual** do dataset.  
- ✅ **Correção de inconsistências** na coluna `maxdebtpduevalodued_3940955A`.  
- ✅ Execução da função para **criação de flags**.  
- ✅ Execução da função para **tratamento de datas**.  
- 💾 Arquivo final salvo em formato **Parquet**.  
